In [1]:
import pandas as pd
import warnings
import pymongo
from datetime import datetime
# pd.set_option('display.max_rows', None)
    
url = "https://www.ecologie.gouv.fr/simulator-energies/monitoring/export/59707a7b55c0012d0efade376d62a56d3c86129a"
df_htt_gas_eur_liter = pd.read_excel(url, sheet_name=0, skiprows=0)
df_htt_gas_eur_liter = df_htt_gas_eur_liter.rename(columns={
    'Gazole €/l htt': 'htt_GAZOLE_eur_liter',
    'Super sans plomb 95 €/l htt': 'htt_SP95_eur_liter',
    'Super SP95 - E10 €/l htt': 'htt_E10_eur_liter',
    'Super sans plomb 98 €/l htt': 'htt_SP98_eur_liter',
    'Superéthanol E85 €/l htt': 'htt_E85_eur_liter',
    'GPL €/l htt': 'htt_GPLC_eur_liter',
})
df_htt_gas_eur_liter['Date'] = pd.to_datetime(df_htt_gas_eur_liter['Date'], dayfirst=True)
print(df_htt_gas_eur_liter.head())

df_ttc_gas_eur_liter = pd.read_excel(url, sheet_name=1, skiprows=0)
df_ttc_gas_eur_liter = df_ttc_gas_eur_liter.rename(columns={
    'Gazole €/l ttc': 'ttc_GAZOLE_eur_liter',
    'Super sans plomb 95 €/l ttc': 'ttc_SP95_eur_liter',
    'Super SP95 - E10 €/l ttc': 'ttc_E10_eur_liter',
    'Super sans plomb 98 €/l ttc': 'ttc_SP98_eur_liter',
    'Superéthanol E85 €/l ttc': 'ttc_E85_eur_liter',
    'GPL €/l ttc': 'ttc_GPLC_eur_liter',
})
df_ttc_gas_eur_liter['Date'] = pd.to_datetime(df_ttc_gas_eur_liter['Date'], dayfirst=True)
print(df_ttc_gas_eur_liter.head())


# Setup MongoDB connection (local)
mongo_host = "localhost"
mongo_port = 27017
mongo_user = "admin"
mongo_password = "password"
auth_db = "admin"
client_mongo = pymongo.MongoClient(
    host=mongo_host,
    port=mongo_port,
    username=mongo_user,
    password=mongo_password,
    authSource=auth_db
)
# ----- Push to MongoDB -----
db_mongo_denorm = client_mongo.get_database("denormalization")

# HTT_gas_eur_liter
db_mongo_denorm.drop_collection("htt_gas_eur_liter")
collection_mongo = db_mongo_denorm.get_collection("htt_gas_eur_liter")
collection_mongo.create_index([("Date", pymongo.ASCENDING)])
records = df_htt_gas_eur_liter.to_dict(orient="records")
collection_mongo.insert_many(records)
print("correctly loaded htt_gas_eur_liter datas to MongoDB in denormalized collection")

# TTC_gas_eur_liter
db_mongo_denorm.drop_collection("ttc_gas_eur_liter")
collection_mongo = db_mongo_denorm.get_collection("ttc_gas_eur_liter")
collection_mongo.create_index([("Date", pymongo.ASCENDING)])
records = df_ttc_gas_eur_liter.to_dict(orient="records")
collection_mongo.insert_many(records)
print("correctly loaded ttc_gas_eur_liter datas to MongoDB in denormalized collection")

        Date  htt_GAZOLE_eur_liter  htt_SP95_eur_liter  htt_E10_eur_liter  \
0 1985-01-04                0.3583                 NaN                NaN   
1 1985-01-11                0.3583                 NaN                NaN   
2 1985-01-18                0.3583                 NaN                NaN   
3 1985-01-25                0.3583                 NaN                NaN   
4 1985-02-01                0.3583                 NaN                NaN   

   htt_SP98_eur_liter  htt_E85_eur_liter  htt_GPLC_eur_liter  
0                 NaN                NaN                 NaN  
1                 NaN                NaN                 NaN  
2                 NaN                NaN                 NaN  
3                 NaN                NaN                 NaN  
4                 NaN                NaN                 NaN  
        Date  ttc_GAZOLE_eur_liter  ttc_SP95_eur_liter  ttc_E10_eur_liter  \
0 1985-01-04                0.6311                 NaN                NaN   
1 198